In [1]:
from langchain_aws import ChatBedrockConverse
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
#DynamoDB를 cpu메모리 대신 사용

In [3]:
#DynamoDB에 메세지를 저장하는 함수
def insert_chat_message(session_id, message_content, timestamp=None):
    """
    Insert a chat message into the DynamoDB table with a timestamp-prefixed message_id.
    
    Args:
        session_id (str): The session identifier
        message_content (str): The content of the message
        role (str): The role of the message sender (e.g., 'user', 'assistant')
        timestamp (str, optional): Message timestamp. Defaults to current time if None.
    
    Returns:
        dict: The response from DynamoDB
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    table = dynamodb.Table('ChatMessageHistory')
    
    # Use current time if timestamp not provided
    if timestamp is None:
        now = datetime.utcnow()
        timestamp = now.isoformat()
    else:
        # If timestamp is provided as string, parse it to datetime
        now = datetime.fromisoformat(timestamp)
    
    # Create a timestamp prefix for message_id in sortable format
    # Format: YYYYMMDD-HHMMSS-microseconds-uuid
    timestamp_prefix = now.strftime("%Y%m%d-%H%M%S-%f")
    
    # Generate a unique message ID with timestamp prefix
    message_id = f"{timestamp_prefix}-{str(uuid.uuid4())[:8]}"
    
    # Prepare item to insert
    item = {
        'session_id': session_id,
        'message_id': message_id,
        'content': message_content,
        'timestamp': timestamp
    }
    
    try:
        # Insert the item into the table
        response = table.put_item(Item=item)
        print(f"Message inserted successfully: {message_id}")
        return response
    except ClientError as e:
        print(f"Error inserting message: {e.response['Error']['Message']}")
        raise


In [4]:
#DynamoDB에 메세지를 지우는 함수
def delete_all_session_messages(session_id):
    """
    Delete all messages for a specific session.
    
    Args:
        session_id (str): The session identifier
    
    Returns:
        dict: Summary of deletion operation
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    table = dynamodb.Table('ChatMessageHistory')
    
    try:
        # First, query to get all messages for the session
        response = table.query(
            KeyConditionExpression=Key('session_id').eq(session_id)
        )
        
        items = response['Items']
        
        # Handle pagination if there are more results
        while 'LastEvaluatedKey' in response:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            items.extend(response['Items'])
        
        # If no items found, return early
        if not items:
            print(f"No messages found for session {session_id}")
            return {"deleted_count": 0}
        
        # Delete each item
        deleted_count = 0
        for item in items:
            table.delete_item(
                Key={
                    'session_id': session_id,
                    'message_id': item['message_id']
                }
            )
            deleted_count += 1
        
        print(f"Deleted {deleted_count} messages for session {session_id}")
        return {"deleted_count": deleted_count}
        
    except ClientError as e:
        print(f"Error deleting messages: {e.response['Error']['Message']}")
        raise


In [5]:
#DynamoDB에 메세지를 가져오는 함수
def get_chat_messages(session_id, ascending=True):
    """
    Retrieve chat messages for a session, sorted by message_id.
    
    Args:
        session_id (str): The session identifier
        ascending (bool): Sort order - True for ascending (oldest first), False for descending (newest first)
    
    Returns:
        list: List of message items
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    table = dynamodb.Table('ChatMessageHistory')
    
    messages = []
    
    try:
        # Query messages for the session, sorted by message_id
        response = table.query(
            KeyConditionExpression=Key('session_id').eq(session_id),
            ScanIndexForward=ascending  # True for ascending, False for descending
        )
        
        # Add results to our messages list
        messages.extend(response['Items'])
        
        # Handle pagination if there are more results
        while 'LastEvaluatedKey' in response:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ScanIndexForward=ascending,  # Maintain the same sort order
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            messages.extend(response['Items'])
        
        print(f"Found {len(messages)} messages for session {session_id}")
        return messages
        
    except ClientError as e:
        print(f"Error retrieving messages: {e.response['Error']['Message']}")
        return []

In [6]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # replace with your region
)
llm = ChatBedrockConverse(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    temperature=0.7,
    max_tokens=2000
)

In [7]:
delete_all_session_messages("bnk-1")

Deleted 1 messages for session bnk-1


{'deleted_count': 1}

In [8]:
get_chat_messages("bnk-1")

Found 0 messages for session bnk-1


[]

In [9]:
template_dynamo = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

Relevant Information:

{chat_history}

Conversation:
Human: {question}
AI:"""
prompt_dynamo  = PromptTemplate(
input_variables=["chat_history", "question"], template=template_dynamo)


In [10]:
chain_dynamo = (
    {
        "seesion_id": itemgetter("seesion_id") ,
        "chat_history": itemgetter("seesion_id") | RunnableLambda(get_chat_messages),
        "question": itemgetter("question"),
    }
    | prompt_dynamo
    | llm
    | StrOutputParser()
)

In [ ]:
chain_dynamo.invoke({"seesion_id" : "bnk-1", "question" : "상사병은 무엇인가요?"})

Found 0 messages for session bnk-1


In [ ]:
insert_chat_message("bnk-1", "상사병은 한사람을 몹시 그리워하고 보고시퍼서 생기는 마음의 병이라고 나는생각해")

In [ ]:
get_chat_messages("bnk-1")

In [ ]:
chain_dynamo.invoke({"seesion_id" : "bnk-1", "question" : "상사병은 무엇인가요?"})